#### CSCE 670 :: Information Storage and Retrieval :: Texas A&M University :: Spring 2018


# Homework 3:  Embeddings + Recommenders

### 100 points [5% of your final grade]

### Due: Monday, April 9 by 11:59pm

*Goals of this homework:* There are two main learning objectives: (i) implement and evaluate a pre-cursor to modern word2vec embeddings; and (ii) implement, evaluate, and improve upon traditional collaborative filtering recommenders.

*Submission Instructions:* To submit your homework, rename this notebook as UIN_hw#.ipynb. For example, this homework submission would be: YourUIN_hw3.ipynb. Submit this notebook via ecampus. Your notebook should be completely self-contained, with the results visible in the notebook. 

*Late submission policy:* For this homework, you may use up to three of your late days, meaning that no submissions will be accepted after Thursday, April 12 at 11:59pm.

# Part 1: Word Embeddings (50 points)
For this first part, we're going to implement a word embedding approach that is a bit simpler than word2vec. The key idea is to look at co-occurrences between center words and context words (somewhat like in word2vec) but without any pesky learning of model parameters.

If you're interested in a deeper treatment of comparing count vs. learned embeddings, take a look at: [Don’t count, predict! A systematic comparison of
context-counting vs. context-predicting semantic vectors](
http://www.aclweb.org/anthology/P14-1023)

## Load the Brown Corpus

The dataset for this part is the (in)famous [Brown corpus](https://en.wikipedia.org/wiki/Brown_Corpus) that is a collection of text samples from a wide range of sources, with over one million unique words. Good for us, you can find the Brown corpus in nltk. *Make sure you have already installed nltk with something like: conda install nltk*

In [234]:
import nltk
nltk.download('brown')
nltk.download('stopwords')

[nltk_data] Downloading package brown to /home/ubuntu/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package stopwords to /home/ubuntu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

Once you have it locally, you can load the dataset into your notebook. You can access the words using brown.words():

In [235]:
from nltk.corpus import brown
brown.words()


[u'The', u'Fulton', u'County', u'Grand', u'Jury', ...]

## 1.1 Dataset Pre-processing
OK, now we need to do some basic pre-processing. For this part you should:

* Remove stopwords and punctuation.
* Make everything lowercase.

Then, count how often each word occurs. We will define the 5,000 most  frequent words as your vocabulary (V). We will define the 1,000 most frequent words as our context (C). Include a print statement below to show the top-20 words after pre-processing.

In [236]:
# Your Code Here...
from nltk.corpus import stopwords
import string
import collections
from collections import Counter

def punctuationCheck(w):
    temp_word = ''.join(char for char in w if char not in set(string.punctuation))
    if len(temp_word) !=0:
        return temp_word
    else:
        return 0
stop_list = stopwords.words('english')
pre_processed = [w.lower() for w in brown.words() if w.lower() not in stop_list]
pre_processed = [punctuationCheck(w) for w in pre_processed if punctuationCheck(w) != 0] 
counts = Counter(pre_processed)
vocabulary = [w[0] for w in counts.most_common(5000)]
context = [w[0] for w in counts.most_common(1000)]
print len(vocabulary)
print len(context)
print "Rank  " +"Word  "+"Count" 
rank = 1
for token, count in counts.most_common(20):
    print str(rank)+"      "+str(token)+"   "+str(count)
    rank += 1

5000
1000
Rank  Word  Count
1      one   3297
2      would   2714
3      said   1961
4      new   1635
5      could   1601
6      time   1598
7      two   1412
8      may   1402
9      first   1361
10      like   1292
11      man   1207
12      even   1170
13      made   1125
14      also   1069
15      many   1030
16      must   1013
17      years   1001
18      af   996
19      back   966
20      well   961


## 1.2 Building the Co-occurrence Matrix 

For each word in the vocabulary (w), we want to calculate how often context words from C appear in its surrounding window of size 4 (two words before and two words after).

In other words, we need to define a co-occurrence matrix that has a dimension of |V|x|C| such that each cell (w,c) represents the number of times c occurs in a window around w. 

In [237]:
# Your Code Here...
import numpy as np
import pandas as pd

In [260]:
#Creating mapping from word to integer 
data_length = len(pre_processed)
def createIndexDictionary(words):
    word_dict = {}
    for i,w in enumerate(words):
        word_dict[w] = i
    return word_dict
#Creating co-occurence matrix
def createMatrix(words,vocab_index,context_index,window):
    coo_matrix = np.zeros((len(vocabulary),len(context)))
    for i in range(data_length):
        w = words[i]
        if w in vocab_index:
            context_window = np.arange(max(i - window,0),min(i + window + 1,data_length))
            context_window = np.setdiff1d(context_window,np.array([1]))
            for j in context_window:
                c = words[j]
                if c in context_index:
                    coo_matrix[vocab_index[w]][context_index[c]] += 1             
    return coo_matrix


In [261]:
vocab_index = createIndexDictionary(vocabulary)
context_index = createIndexDictionary(context)
coo_matrix = createMatrix(pre_processed,vocab_index,context_index,2)
print coo_matrix

[[3.381e+03 7.600e+01 5.200e+01 ... 0.000e+00 1.000e+00 4.000e+00]
 [7.600e+01 2.784e+03 6.900e+01 ... 1.000e+00 1.000e+00 2.000e+00]
 [5.200e+01 6.900e+01 1.999e+03 ... 2.000e+00 0.000e+00 0.000e+00]
 ...
 [0.000e+00 0.000e+00 0.000e+00 ... 0.000e+00 0.000e+00 0.000e+00]
 [0.000e+00 0.000e+00 0.000e+00 ... 0.000e+00 0.000e+00 0.000e+00]
 [0.000e+00 0.000e+00 0.000e+00 ... 0.000e+00 0.000e+00 0.000e+00]]


## 1.3 Probability Distribution

Using the co-occurrence matrix, we can compute the probability distribution Pr(c|w) of context word c around w as well as the overall probability distribution of each context word c with Pr(c).  

In [263]:
# Your Code Here..
def getProbDist(coo_matrix):
    prob_matrix = np.divide(coo_matrix.T,coo_matrix.sum(axis = 1))
    return prob_matrix

prob_matrix = getProbDist(coo_matrix)
print prob_matrix.shape
print "Pr(c|w) for each context word in co-occurence matrix"
print prob_matrix

(1000, 5000)
Pr(c|w) for each c,w combination in co-occurence matrix
[[3.49601903e-01 9.59353699e-03 9.32066679e-03 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [7.85854617e-03 3.51426407e-01 1.23678079e-02 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [5.37690001e-03 8.70992174e-03 3.58307940e-01 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 ...
 [0.00000000e+00 1.26230750e-04 3.58487184e-04 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [1.03401923e-04 1.26230750e-04 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [4.13607693e-04 2.52461500e-04 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]]


In [264]:
def getOverallProb(coo_matrix):
    overall_counts = ((coo_matrix).sum(axis = 0))
    return np.divide(overall_counts,overall_counts.sum())

overall_probabilities = getOverallProb(coo_matrix)
print "Pr(c) for each context word in co-occurence matrix"
print overall_probabilities

Pr(c) for each context word in co-occurence matrix
[0.01386936 0.01153019 0.00812653 0.00693986 0.00690154 0.00690672
 0.00593543 0.00595096 0.0058557  0.00516192 0.00511118 0.00481814
 0.00465349 0.00437495 0.00439255 0.00437288 0.00442672 0.00425379
 0.00408501 0.00412229 0.00404877 0.00384788 0.00371845 0.00323384
 0.00358073 0.00349064 0.00349789 0.00348132 0.00347925 0.00337777
 0.00325558 0.00335085 0.00324212 0.00324833 0.00314893 0.00332392
 0.00304227 0.00301017 0.00290352 0.00287866 0.00302881 0.00293354
 0.00286106 0.00275648 0.0026581  0.00288695 0.00289523 0.00265293
 0.00262186 0.00273887 0.00257526 0.0025939  0.00247586 0.00270367
 0.00257319 0.00247379 0.00245308 0.00240752 0.00244169 0.00241683
 0.00227394 0.00233814 0.00232882 0.00235781 0.00225633 0.0022087
 0.00193119 0.00222113 0.00211447 0.00215278 0.00213415 0.00208548
 0.0021445  0.00202956 0.00213207 0.00212483 0.00206995 0.00215796
 0.00199539 0.00200057 0.00196743 0.0019664  0.0019985  0.00190944
 0.0019519  

## 1.4 Embedding Representation

Now you can represent each vocabulary word as a |C| dimensional vector using this equation:

Vector(w)= max(0, log (Pr(c|w)/Pr(c)))

This is a traditional approach called *pointwise mutual information* that pre-dates word2vec by some time. 

In [265]:
# Your Code Here...
def getPMIMatrix(prob_matrix,overall_probabilities):
    pmi_matrix =  np.divide(prob_matrix.T,overall_probabilities)
    pmi_matrix = np.maximum(np.zeros((len(vocabulary), len(context))), np.log(pmi_matrix))
    print pmi_matrix.shape
    return pmi_matrix
pmi_matrix = getPMIMatrix(prob_matrix,overall_probabilities)
%store pmi_matrix
print pmi_matrix

(5000, 1000)
Stored 'pmi_matrix' (ndarray)
[[3.22711311 0.         0.         ... 0.         0.         0.06846437]
 [0.         3.41703189 0.06932895 ... 0.         0.         0.        ]
 [0.         0.07012853 3.78625893 ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]


/home/ubuntu/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


## 1.5 Analysis

So now we have some embeddings for each word. But are they meaningful? For this part, you should:

- First, cluster the vocabulary into 100 clusters using k-means. Look over the words in each cluster, can you see any relation beween words? Discuss your observations.

- Second, for the top-20 most frequent words, find the nearest neighbors using cosine distance (1- cosine similarity). Do the findings make sense? Discuss.

In [253]:
%store -r pmi_matrix

In [266]:
# First part
#K-Means clustering
from sklearn.cluster import KMeans
kmeans_model = KMeans(n_clusters=100,max_iter=300)
kmeans_model.fit(pmi_matrix)
centroids = kmeans_model.cluster_centers_.argsort(axis=1)[:,::-1]
reverse_vocab_index = {w:i for i,w in vocab_index.items()}
for i in range(10):
    print "\n****** Cluster - "+ str(i+1) + " ******"
    for c in centroids[i, :10]:
        print reverse_vocab_index[c]


****** Cluster - 1 ******
2
1
number
three
two
four
3
used
5
10

****** Cluster - 2 ******
principle
analysis
latter
established
aj
subject
followed
best
filled
mass

****** Cluster - 3 ******
understanding
moral
sound
involved
basic
deal
support
communist
included
applied

****** Cluster - 4 ******
ones
bad
events
pattern
wife
least
using
democratic
particular
tax

****** Cluster - 5 ******
across
around
back
walked
door
street
behind
front
house
car

****** Cluster - 6 ******
covered
due
heat
space
poet
principle
reasons
horse
negro
blood

****** Cluster - 7 ******
town
council
hall
property
afternoon
sunday
meeting
hospital
charles
near

****** Cluster - 8 ******
claim
court
sides
station
sun
ahead
speak
determine
feed
theory

****** Cluster - 9 ******
secretary
commission
section
shall
provided
act
7
states
respect
made

****** Cluster - 10 ******
tax
income
paid
property
pay
return
whether
government
due
money


### Discussion
The words in some clusters are more semantically related as comapred to others.
- Cluster 1 points to a relationship feature of numbers.
- Cluster 5 points to a relationship feature of streets or roads
- Cluster 10 points to a relationship feature of money

In [259]:
# Second part
#Nearest neighbours
%store -r pmi_matrix
from sklearn.neighbors import NearestNeighbors
nn_model = NearestNeighbors(n_neighbors=10, metric = 'cosine')
nn_model.fit(pmi_matrix)
nearest_neighbours = nn_model.kneighbors(pmi_matrix[:20,], n_neighbors=10)
reverse_context_index = {w:i for i,w in context_index.items()}
for i in range(20):
    print "\n****** Word - "+str(reverse_vocab_index.get(i))+" ******"
    for n in nearest_neighbours[1][i, :]:
        print reverse_context_index.get(n)



****** Word - one ******
one
thing
another
every
hundred
thousand
None
least
None
expect

****** Word - would ******
would
could
want
might
expect
cant
ask
think
say
anything

****** Word - said ******
said
oh
im
know
yes
think
tell
hell
thats
ill

****** Word - new ******
new
york
city
central
england
churches
None
development
members
plant

****** Word - could ******
could
hear
see
would
anything
tell
want
ever
else
cant

****** Word - time ******
time
short
long
period
spent
first
much
day
length
hours

****** Word - two ******
two
three
several
four
ago
six
many
couple
weeks
five

****** Word - may ******
may
also
shall
amount
seem
result
necessary
might
likely
cause

****** Word - first ******
first
second
next
appeared
last
time
steps
final
None
two

****** Word - like ******
like
look
looked
id
know
felt
hell
woman
think
tell

****** Word - man ******
man
young
old
woman
god
wife
dead
girl
love
big

****** Word - even ******
even
though
seem
greater
perhaps
might
something
much

### Discussion
Yes, the findings do make sense and are better than the ones obtained in case of kmeans clustering.
Words like 'would','time','two','years' and 'well' have the most meaningful neighbours closely related to the specific word. However, the rest of the words are not as strongly related semantically as compared to the other words.

# Part 2. Collaborative Filtering (50 points)

In this second part, you will implement collaborative filtering on the Netflix prize dataset -- don’t freak out, the provided sample dataset has only ~2000 items and ~28,000 users.

As background, read the paper [Empirical Analysis of Predictive Algorithms for Collaborative Filtering](https://arxiv.org/pdf/1301.7363.pdf) up to Section 2.1. Of course you can read further if you are interested, and you can also refer to the course slides for collaborative filtering.

## 2.1 Load Netflix Data

The dataset is subset of movie ratings data from the Netflix Prize Challenge. Download the dataset from Piazza. It contains a train set, test set, movie file, and README file. The last two files are original ones from the Netflix Prize, however; in this homework you will deal with train and test files which both are subsets of the Netflix training data. Each of train and test files has lines having this format: MovieID,UserID,Rating.

Your job is to predict a rating in the test set using those provided in the training set.

In [204]:
# load the data, then print out the number of ratings, 
# movies and users in each of train and test sets.
# Your Code Here...
from IPython.display import display
import pandas as pd
training_data = pd.read_csv('netflix-dataset/TrainingRatings.txt', sep=",", names = ['MovieID','UserID','Rating'])
testing_data = pd.read_csv('netflix-dataset/TestingRatings.txt', sep=",", names = ['MovieID','UserID','Rating'])
print "Training Data"
training_users = training_data.UserID.unique()
training_movies = training_data.MovieID.unique()
tr_users = training_users.shape[0]
tr_movies = training_movies.shape[0]
print str(tr_users)+" Users"
print str(tr_movies)+" Movies"
display(training_data)
training_data.to_csv('training.csv')
print "Testing Data"
testing_users = testing_data.UserID.unique()
testing_movies = testing_data.MovieID.unique()
te_users = testing_users.shape[0]
te_movies = testing_movies.shape[0]
print str(testing_data.UserID.unique().shape[0])+" Users"
print str(testing_data.MovieID.unique().shape[0])+" Movies"
display(testing_data)
testing_data.to_csv('testing.csv')

Training Data
28978 Users
1821 Movies


,MovieID,UserID,Rating
0,8,1744889,1.0
1,8,1395430,2.0
2,8,1205593,4.0
3,8,1488844,4.0
4,8,1447354,1.0
5,8,306466,4.0
6,8,1331154,4.0
7,8,1818178,3.0
8,8,991725,4.0
9,8,1987434,4.0


Testing Data
27555 Users
1701 Movies


,MovieID,UserID,Rating
0,8,573364,1.0
1,8,2149668,3.0
2,8,1089184,3.0
3,8,2465894,3.0
4,8,534508,1.0
5,8,992921,4.0
6,8,595054,4.0
7,8,1298304,4.0
8,8,1661600,4.0
9,8,553787,2.0


## 2.2 Implement CF

In this part, you will implement the basic collaborative filtering algorithm described in Section 2.1 of the paper -- that is, focus only on Equations 1 and 2 (where Equation 2 is just the Pearson correlation). You should consider the first 5,000 users with their associated items in the test set. 

Note that you should test the algorithm for a small set of users e.g., 10 users first and then run for 5,000 users. It may take long to run but you won't have memory issues. 

Set k to 0.1. 

In [ ]:
import numpy as np
#Creates a item to index mapping for items (maybe users or movies)
def createNetflixIndexDictionary(items):
    index_dict = {}
    for i,item in enumerate(items):
        index_dict[item] = i
    return index_dict

In [ ]:
tr_user_index = createNetflixIndexDictionary(training_users)
tr_movie_index = createNetflixIndexDictionary(training_movies) 
te_user_index = createNetflixIndexDictionary(testing_users)
te_movie_index = createNetflixIndexDictionary(testing_movies) 

In [ ]:
#Creates the rating matrix for the 
def createRatingMatrix(data,user_index,movie_index):
    temp_matrix = np.zeros((len(user_index),len(movie_index))) 
    for record in data.itertuples():
        ui = user_index.get(record[2])
        mi = movie_index.get(record[1])
        temp_matrix.itemset((ui,mi),record[3])
    return temp_matrix

In [ ]:
#Generates a dictionary indexed by user with their corresponding list of movies 
def createUserMovieDictionary(user_index,movie_index,data_matrix):
    user_movies_dict = {}
    for user in user_index:
        ui = user_index.get(user)
        movies = []
        for movie in movie_index:
            mi = movie_index.get(movie)
            if data_matrix[ui,mi] != 0.0:
                movies.append(movie)
        user_movies_dict[user] = movies
    return user_movies_dict

In [ ]:
training_matrix = createRatingMatrix(training_data,tr_user_index,tr_movie_index)

In [ ]:
testing_matrix = createRatingMatrix(testing_data,te_user_index,te_movie_index)

In [ ]:
tr_user_movie = createUserMovieDictionary(tr_user_index,tr_movie_index,training_matrix)

In [ ]:
te_user_movie = createUserMovieDictionary(te_user_index,te_movie_index,testing_matrix)

In [ ]:
import math
# from scipy.stats import pearsonr

def calculateAvgRating(user,user_index,data_matrix):
    ui = user_index.get(user)
    return np.mean(data_matrix[ui])
#Creates Average Rating Dictionary
def createAvgRatingDictionary(users,user_index,matrix):
    avg_dict = {}
    for user in users:
        avg_dict[user] = calculateAvgRating(user,user_index,matrix)
    return avg_dict

In [ ]:
tr_avg_dict = createAvgRatingDictionary(training_users,tr_user_index,training_matrix)

In [ ]:
te_avg_dict = createAvgRatingDictionary(testing_users,te_user_index,testing_matrix)

In [ ]:
def createCorrelationDictonary(start,end):
    corr_dict = {}
    num_rating = 0
    denom_sum1 = 0
    denom_sum2 = 0
    for user1 in testing_users[start:end]:
#         u1_ratings = []
#         u2_ratings = []
        corr_dict[user1] = {}
        for user2 in training_users:
            if user1 != user2:
                common_movies = np.intersect1d(te_user_movie[user1],tr_user_movie[user2])
                if common_movies.size != 0:
                    for movie in common_movies:
                        m1i = te_movie_index.get(movie)
                        m2i = tr_movie_index.get(movie)
                        u1i = te_user_index.get(user1)
                        u2i = tr_user_index.get(user2)
                        te = testing_matrix[u1i,m1i] - te_avg_dict[user1]
                        tr = training_matrix[u2i,m2i] - tr_avg_dict[user2]
                        num_rating += round(((te)*(tr)),2)
                        denom_sum1 += round(pow((te),2),2)
                        denom_sum2 += round(pow((tr),2),2)
#                         u1_ratings.append(testing_matrix[u1i,m1i])
#                         u2_ratings.append(training_matrix[u2i,m2i])
#                     corr_dict[user1][user2] = np.corrcoef(u1_ratings,u2_ratings)
                    corr_dict[user1][user2] = round(num_rating/math.sqrt(float(denom_sum1*denom_sum2)),2) 
    return corr_dict

In [270]:
def createDataframe(number_of_users):
    df = pd.DataFrame()
    print "Elements done",
    for i in range(0,number_of_users,100):
        print i,
        temp_df = pd.DataFrame.from_dict(createCorrelationDictonary(i,i+100),'index')
        df = pd.concat([df,temp_df])
    return df

In [ ]:
df = createDataframe(5000)

In [233]:
print "Correlation Dataframe"
display(df)

Correlation Dataframe


,7,79,199,481,769,906,1310,1333,1427,1442,...,2648572,2648589,2648730,2648734,2648853,2648869,2648885,2649120,2649267,2649285
9660,0.91,NaN,0.91,NaN,0.91,0.91,NaN,0.90,0.91,NaN,...,0.91,0.91,NaN,NaN,NaN,0.91,NaN,0.91,NaN,NaN
112790,0.91,0.92,0.92,0.92,0.92,0.92,0.92,0.91,0.92,0.92,...,0.92,0.91,0.92,0.92,0.92,0.92,0.92,0.92,0.92,0.92
152955,0.92,0.92,0.92,0.92,0.92,0.92,0.92,0.92,0.92,0.92,...,0.92,0.92,0.92,0.92,0.92,0.92,0.92,0.92,0.92,0.92
155279,0.91,0.91,0.91,0.91,NaN,0.91,0.91,0.91,0.91,0.91,...,0.91,0.91,0.91,0.91,0.91,0.91,0.91,0.91,NaN,0.91
157806,0.92,0.92,0.92,NaN,NaN,NaN,NaN,0.92,0.92,0.92,...,NaN,NaN,0.92,NaN,NaN,0.92,0.92,NaN,NaN,NaN
254671,0.90,0.91,0.90,0.90,0.90,NaN,0.90,0.90,0.91,NaN,...,0.91,0.90,NaN,0.90,0.90,0.91,NaN,NaN,0.90,0.90
264988,0.92,0.92,0.92,0.92,0.92,0.92,0.92,0.92,0.92,0.92,...,0.92,0.92,0.92,0.92,0.92,0.92,0.92,0.92,0.92,0.92
320540,0.91,NaN,NaN,NaN,NaN,NaN,NaN,0.91,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.91,NaN,NaN,0.91,NaN
394189,0.92,0.92,0.92,0.92,0.92,0.92,0.92,0.92,0.92,0.92,...,0.92,0.92,0.92,0.92,0.92,0.92,0.92,0.92,0.92,0.92
400756,0.92,NaN,0.92,NaN,0.92,0.92,NaN,0.92,0.92,0.92,...,NaN,0.92,0.92,NaN,NaN,0.92,NaN,0.92,NaN,NaN


In [ ]:
#Calculating K considering normalized weights (as 0.1 was giving values out of range)
def calculateK(movie,active_user):
    check_movie = training_data['MovieID'] == movie
    user_list = (training_data[check_movie]['UserID'])
    sum_p = 0.0
    k = 0.0
    for user in user_list:
        if (not (df.at[active_user,user] == np.nan)):
            sum_p += abs(df.at[active_user,user])
    if sum_p == 0.0:
        k = 0.0
    else:
        k = 1/float(sum_p)    
    return k
#Calculating Predicted Rating for a user and a movie
def predictRating(active_user,movie):
    active_avg = calculateAvgRating(active_user,te_user_index,testing_matrix)
    sum_rating = 0.0
    mi = tr_movie_index.get(movie)
    for user in tr_user_index:
        ui =  tr_user_index.get(user)
        if (not math.isnan(df.at[active_user,user])):
            pearson_c = df.at[active_user,user]
            sum_rating += (pearson_c)*(training_matrix[ui,mi] - tr_avg_dict[user])
    k = calculateK(movie,active_user)
#     k = 0.0001
    prediction  = active_avg + k*(sum_rating)
    if prediction > 5.0:
        prediction = 5.0
    elif prediction < 1.0:
        prediction = 1.0
    return float(int(round(prediction)))

In [ ]:
#Calculating Predictions for a subset of the test set (start and end comprise the index of the number of users tested)
def predictionForTestSet(start,end):
    test_predictions={}
    for user in testing_users[start:end]:
        user_movies = te_user_movie[user]
        user_predictions = []
        for movie in user_movies:
            user_predictions.append(predictRating(user,movie))
        test_predictions[user] = user_predictions
    return test_predictions
        
        

In [ ]:
#Creating a dataframe from the predictions to reduce disk space usage
def createPredictionsDataframe(number_of_users):
    predictions_df = pd.DataFrame()
    print "Elements done",
    for i in range(0,number_of_users,1):
        print i,
        temp_pred_df = pd.DataFrame.from_dict(predictionForTestSet(i,i+1),'index')
        predictions_df = pd.concat([predictions_df,temp_pred_df])
    display(predictions_df)
    return predictions_df

In [ ]:
%%time
predictions_df = createPredictionsDataframe(5000)

In [164]:
%store predictions_df

Stored 'predictions_df' (DataFrame)


In [248]:
print "Predictions for the 5000 users and their corresponding movies"
display(predictions_df)

Predictions for the 5000 users and their corresponding movies


,0,1,2,3,4,5,6,7,8,9,...,60,61,62,63,64,65,66,67,68,69
573364,1.0,1.0,3.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2149668,1.0,3.0,3.0,3.0,2.0,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1089184,2.0,2.0,3.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2465894,1.0,2.0,1.0,1.0,3.0,2.0,4.0,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
534508,1.0,3.0,1.0,1.0,3.0,2.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
992921,2.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
595054,1.0,3.0,3.0,1.0,1.0,3.0,3.0,2.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1298304,3.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1661600,1.0,1.0,3.0,3.0,3.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
553787,2.0,2.0,1.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 2.3 Evaluation 

You should evaluate your predictions using Mean Absolute Error and Root Mean Squared Error. 

In [218]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
def getGroundTruth(number_of_users):
    test_gt={}
    for user in testing_users[:number_of_users]:
        ui = te_user_index.get(user)
        user_movies = te_user_movie[user]
        user_gt = []
        for movie in user_movies:
            mi = te_movie_index.get(movie)
            user_gt.append(testing_matrix[ui,mi])
        test_gt[user] = user_gt
    return test_gt
def meanAbsoluteError(number_of_users,test_predictions,test_gt):
    mae_list = [] 
    #Converting dataframe to dictionary
    test_predictions = test_predictions.to_dict('index')
    for user in testing_users[:number_of_users]:
        if user in test_gt:
            #Removing nan from the predictions
            predictions = [x for x in test_predictions[user].values() if not np.isnan(x)]
            mae = mean_absolute_error(test_gt.get(user),predictions)
            mae_list.append(mae)
    return mae_list
def rootMeanSquaredError(number_of_users,test_predictions,test_gt):
    rmse_list = []
    #Converting dataframe to dictionary
    test_predictions = test_predictions.to_dict('index')
    for user in testing_users[:number_of_users]:
        if user in test_gt:
            #Removing nan from the predictions
            predictions = [x for x in test_predictions[user].values() if not np.isnan(x)]
            rmse = math.sqrt(mean_squared_error(test_gt.get(user),predictions))
            rmse_list.append(rmse)
    return rmse_list

In [220]:
test_gt = getGroundTruth(5000)
mae_list = meanAbsoluteError(5000,predictions_df,test_gt)
rmse_list = rootMeanSquaredError(5000,predictions_df,test_gt)

In [231]:
print "Mean Absolute Error for the first 20 users of the test set\n"
print mae_list[:20]
print "\n"
print "Average MAE for all the 5000 users:",
print np.mean(mae_list)
print
# print len(mae_list)
print "Root Mean Squared Error for the first 20 users of the test set\n"
print rmse_list[:20]
print "\n"
print "Average RMSE for all the 5000 users:",
print np.mean(rmse_list)
# print len(rmse_list)

Mean Absolute Error for the first 20 users of the test set

[0.5, 1.1666666666666667, 1.25, 1.25, 0.5, 1.5, 1.0, 0.5, 1.6, 1.0, 1.2, 0.48, 2.0, 2.0, 1.4, 0.6666666666666666, 1.5, 2.0, 1.6666666666666667, 0.8333333333333334]


Average MAE for all the 5000 users: 1.1797965981477454

Root Mean Squared Error for the first 20 users of the test set

[1.0, 1.6832508230603465, 1.5, 1.5, 0.9128709291752769, 1.5811388300841898, 1.5, 0.7071067811865476, 2.280350850198276, 1.224744871391589, 1.5491933384829668, 0.8944271909999159, 2.23606797749979, 2.23606797749979, 2.04939015319192, 0.816496580927726, 1.5811388300841898, 2.0, 2.0816659994661326, 1.224744871391589]


Average RMSE for all the 5000 users: 1.4244487068630984


## 2.4 Extensions

Given your results in the previous part, can you do better? For this last part you should report on your best attempt at improving MAE and RMSE. Provide code, results, plus a brief discussion on your approach.

#### Default Rating
In this approach, we assign a default rating to each movie that the active user or the user in the training set have not rated and find the correlation over the union of the movies voted between the active user and the training user we are finding correlation with. The benefits of default rating are as follows:
- Allowing for correlation over the union of movies
- Can allow to add additonal movies (unrated by either user) with default ratings and calculate correlation assuming they agree upon them
- Can help in implicit feedback (if we assume a rating of 0.0 for the unrated items) as we extend the user data by assuming about the unrated movies.

In [267]:
# Your Code Here...
#Default Rating on a smaller subset of 100 users
#In this case, we remove the check for zero ratings while creating the user and movie index
def createDefaultRatingUserMovieDictionary(user_index,movie_index,data_matrix):
    user_movies_dict = {}
    for user in user_index:
        ui = user_index.get(user)
        movies = []
        for movie in movie_index:
            movies.append(movie)
        user_movies_dict[user] = movies
    return user_movies_dict
tr_user_movie_default = createDefaultRatingUserMovieDictionary(tr_user_index,tr_movie_index,training_matrix)
te_user_movie_default = createDefaultRatingUserMovieDictionary(te_user_index,te_movie_index,training_matrix)

In [ ]:
#Creating correlation dictionary for 100 users
default_df = createDataframe(100)
def calculateKDefault(movie,active_user):
    check_movie = training_data['MovieID'] == movie
    user_list = (training_data[check_movie]['UserID'])
    sum_p = 0.0
    k = 0.0
    for user in user_list:
        if (not (default_df.at[active_user,user] == np.nan)):
            sum_p += abs(default_df.at[active_user,user])
    if sum_p == 0.0:
        k = 0.0
    else:
        k = 1/float(sum_p)    
    return k
def predictRatingDefault(active_user,movie):
    active_avg = calculateAvgRating(active_user,te_user_index,testing_matrix)
    sum_rating = 0.0
    mi = tr_movie_index.get(movie)
    for user in tr_user_index:
        ui =  tr_user_index.get(user)
        if (not math.isnan(default_df.at[active_user,user])):
            pearson_c = default_df.at[active_user,user]
            sum_rating += (pearson_c)*(training_matrix[ui,mi] - tr_avg_dict[user])
    k = calculateKDefault(movie,active_user)
#     k = 0.0001
    prediction  = active_avg + k*(sum_rating)
    if prediction > 5.0:
        prediction = 5.0
    elif prediction < 1.0:
        prediction = 1.0
    return float(int(round(prediction)))
def predictionForTestSetDefault(start,end):
    test_predictions={}
    for user in testing_users[start:end]:
        user_movies = te_user_movie[user]
        user_predictions = []
        for movie in user_movies:
            user_predictions.append(predictRatingDefault(user,movie))
        test_predictions[user] = user_predictions
    return test_predictions
predictions_default = predictionForTestSetDefault(0,100)

In [275]:
def defaultValueModifications(number_of_users,predictions):
    test_gt = getGroundTruth(number_of_users)
    mae_list = meanAbsoluteError(number_of_users,predictions,test_gt)
    rmse_list = rootMeanSquaredError(number_of_users,predictions,test_gt)
    return (mae_list,rmse_list)
predictions_default = pd.DataFrame.from_dict(predictions_default,'index')
default = defaultValueModifications(100,predictions_default)
mae_list_d = default[0]
rmse_list_d = default[1]
print "Mean Absolute Error for the first 20 users of the test set\n"
print mae_list_d[:20]
print "\n"
print "Average MAE for all the 5000 users:",
print np.mean(mae_list_d)
print
# print len(mae_list)
print "Root Mean Squared Error for the first 20 users of the test set\n"
print rmse_list_d[:20]
print "\n"
print "Average RMSE for all the 5000 users:",
print np.mean(rmse_list_d)

Mean Absolute Error for the first 20 users of the test set

[0.5, 1.1666666666666667, 1.25, 1.25, 0.5, 1.5, 1.0, 0.5, 1.6, 1.0, 1.2, 0.48, 2.0, 2.0, 1.4, 0.6666666666666666, 1.5, 2.0, 1.6666666666666667, 0.8333333333333334]


Average MAE for all the 5000 users: 1.253042784992785

Root Mean Squared Error for the first 20 users of the test set

[1.0, 1.6832508230603465, 1.5, 1.5, 0.9128709291752769, 1.5811388300841898, 1.5, 0.7071067811865476, 2.280350850198276, 1.224744871391589, 1.5491933384829668, 0.8944271909999159, 2.23606797749979, 2.23606797749979, 2.04939015319192, 0.816496580927726, 1.5811388300841898, 2.0, 2.0816659994661326, 1.224744871391589]


Average RMSE for all the 5000 users: 1.539506720556609


#### Inverse User Frequencies
In this approach, we use the intuition that movies which are more popular (have the most number of ratings by the user) are less informative of a user's preference. Thus, universally liked items in the intersection of items liked by the active user and the training set users are not useful in capturing the similarity between users as an intersection of rarely liked items are. Thus, as given in the paper, we can define an inverse frequency factor $f_{j} = \log{\frac{n}{n_{j}}}$ where $n$ is the total number of users in the training set and $n_{j}$ is the number of users who have rated movie $j$.

In [304]:
#Inverse User Frequencies
def calculateFrequencyFactor(movie):
    check_movie = training_data['MovieID'] == movie
    user_list = (training_data[check_movie]['UserID'])
    return math.log(tr_users/len(user_list))
def getFrequencyDictionaryAndSum():
    freq_dict = {}
    sum = 0.0
    for movie in training_movies:
        f = calculateFrequencyFactor(movie)
        freq_dict[movie] = f
        sum += f
    return (freq_dict,sum)
def createCorrelationDictonary_IF(start,end):
    corr_dict = {}
    t1 = 0.0
    t2 = 0.0
    t3 = 0.0
    t4 = 0.0
    t5 = 0.0
    num_rating = 0
    denom_sum1 = 0
    denom_sum2 = 0
    fandsum = getFrequencyDictionaryAndSum()
    f_dict = fandsum[0]
    f_sum = fandsum[1]
    for user1 in testing_users[start:end]:
        corr_dict[user1] = {}
        for user2 in training_users:
            if user1 != user2:
                common_movies = np.intersect1d(te_user_movie[user1],tr_user_movie[user2])
                if common_movies.size != 0:
                    for movie in common_movies:
                        m1i = te_movie_index.get(movie)
                        m2i = tr_movie_index.get(movie)
                        u1i = te_user_index.get(user1)
                        u2i = tr_user_index.get(user2)
                        
                        t1 += (testing_matrix[u1i,m1i]*training_matrix[u2i,m2i]*f_dict[movie])
                        t2 += (f_dict[movie]*testing_matrix[u1i,m1i])
                        t3 += (f_dict[movie]*training_matrix[u2i,m2i])
                        t4 += (f_dict[movie]*pow(testing_matrix[u1i,m1i],2))
                        t5 += (f_dict[movie]*pow(training_matrix[u2i,m2i],2))
                        num_rating = (f_sum*(t1) - ((t2)*(t3)))
                        denom_sum1 = f_sum*(t4 - (t2**2))
                        denom_sum2 = f_sum*(t5 - (t3**2))
                    corr_dict[user1][user2] = round(num_rating/math.sqrt(float(denom_sum1*denom_sum2)),2) 
    return corr_dict
def calculateK_IF(movie,active_user):
    check_movie = training_data['MovieID'] == movie
    user_list = (training_data[check_movie]['UserID'])
    sum_p = 0.0
    k = 0.0
    for user in user_list:
        if (not (df_if.at[active_user,user] == np.nan)):
            sum_p += abs(df_if.at[active_user,user])
    if sum_p == 0.0:
        k = 0.0
    else:
        k = 1/float(sum_p)    
    return k
def createDataframe_IF(number_of_users):
    df_if = pd.DataFrame()
    print "Elements done",
    for i in range(0,number_of_users,100):
        print i,
        temp_df = pd.DataFrame.from_dict(createCorrelationDictonary_IF(i,i+100),'index')
        df_if = pd.concat([df_if,temp_df])
    return df_if
df_if = createDataframe_IF(100)
def predictRating_IF(active_user,movie):
    active_avg = calculateAvgRating(active_user,te_user_index,testing_matrix)
    sum_rating = 0.0
    mi = tr_movie_index.get(movie)
    for user in tr_user_index:
        ui =  tr_user_index.get(user)
        if (not math.isnan(df_if.at[active_user,user])):
            pearson_c = df_if.at[active_user,user]
            sum_rating += (pearson_c)*(training_matrix[ui,mi] - tr_avg_dict[user])
    k = calculateK_IF(movie,active_user)
#     k = 0.0001
    prediction  = active_avg + k*(sum_rating)
    if prediction > 5.0:
        prediction = 5.0
    elif prediction < 1.0:
        prediction = 1.0
    return float(int(round(prediction)))
#Creating correlation dictionary for 100 users
def predictionForTestSet_IF(start,end):
    test_predictions={}
    for user in testing_users[start:end]:
        user_movies = te_user_movie[user]
        user_predictions = []
        for movie in user_movies:
            user_predictions.append(predictRatingDefault(user,movie))
        test_predictions[user] = user_predictions
    return test_predictions
predictions_if = predictionForTestSet_IF(0,100)
#Different Weighing Strategies

#Spearman Correlation

 Elements done 0


In [288]:
display(predictions_if)

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
803330,1.0,1.0,3.0,2.0,4.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1656839,1.0,3.0,4.0,1.0,3.0,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
320540,2.0,2.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1539617,1.0,2.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1922607,1.0,3.0,3.0,3.0,3.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2170930,2.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2051123,2.0,1.0,3.0,4.0,2.0,1.0,2.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2554933,1.0,3.0,1.0,3.0,3.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1695799,1.0,2.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2455107,2.0,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [289]:
def ifModifications(number_of_users,predictions):
    test_gt = getGroundTruth(number_of_users)
    mae_list = meanAbsoluteError(number_of_users,predictions,test_gt)
    rmse_list = rootMeanSquaredError(number_of_users,predictions,test_gt)
    return (mae_list,rmse_list)
# predictions_if = pd.DataFrame.from_dict(predictions_if,'index')
if_lists = ifModifications(100,predictions_if)
mae_list_i = if_lists[0]
rmse_list_i = if_lists[1]
print "Mean Absolute Error for the first 20 users of the test set\n"
print mae_list_i[:20]
print "\n"
print "Average MAE for all the 5000 users:",
print np.mean(mae_list_i)
print
# print len(mae_list)
print "Root Mean Squared Error for the first 20 users of the test set\n"
print rmse_list_i[:20]
print "\n"
print "Average RMSE for all the 5000 users:",
print np.mean(rmse_list_i)

Mean Absolute Error for the first 20 users of the test set

[0.5, 1.1666666666666667, 1.25, 1.25, 0.5, 1.5, 1.0, 0.5, 1.6, 1.0, 1.2, 0.48, 2.0, 2.0, 1.4, 0.6666666666666666, 1.5, 2.0, 1.6666666666666667, 0.8333333333333334]


Average MAE for all the 5000 users: 1.253042784992785

Root Mean Squared Error for the first 20 users of the test set

[1.0, 1.6832508230603465, 1.5, 1.5, 0.9128709291752769, 1.5811388300841898, 1.5, 0.7071067811865476, 2.280350850198276, 1.224744871391589, 1.5491933384829668, 0.8944271909999159, 2.23606797749979, 2.23606797749979, 2.04939015319192, 0.816496580927726, 1.5811388300841898, 2.0, 2.0816659994661326, 1.224744871391589]


Average RMSE for all the 5000 users: 1.539506720556609


#### Significance Weighting
More emphasis can be applied to weights that are closer to one and the effect of low weights can be penalized.

In [ ]:
#The following code can be used to add variable significance to weights
def addingWeighingSignificance(df):
    check_greater = ((1-df) < 0.05)
    new_df = df[check_greater] 
    new_df.replace(new_df,-1) 
    return new_df

### Other Improvements
#### Spearman Correlation
The Spearman correlation evaluates the monotonic relationship between two continuous or ordinal variabes where the variables tend to change together but not at constant rate. The Spearman correlation is based on the ranked values of each variable rather than raw data. Since, the rating of a movie may be considered an ordinal data type, weight calculation using Spearman correlation can be more useful as compared to Pearson correlation.

#### Context Aware
This strategy makes use of other information available to compute the similarity. For example, ratings might vary depending on speical holidays, or ratings might change depending on the preferred genre of the user.


### References
https://pythonprogramminglanguage.com/kmeans-text-clustering/

http://scikit-learn.org/stable/auto_examples/text/document_clustering.html

https://pandas.pydata.org/pandas-docs/stable/